In [ ]:
# @title
def setup_tecton_magic():
    """Setup the %tecton cell magic"""
    from IPython.core.magic import register_line_cell_magic

    @register_line_cell_magic
    def tecton(line):
        from shlex import split
        from tecton_core import conf
        from tecton.cli import cli as tecton_cli
        from tecton.identities.credentials import who_am_i, ServiceAccountProfile

        args = split(line)
        # Special case "login" to support interactive authentication.
        if args and args[0] == "login":
            # I'm not sure what to call this flag.  "--quiet" didn't seem to fit and "if-needed" had weird highlighting.
            if not ("--when-needed" in args and who_am_i()):
                tecton_login = tecton_cli.login.callback.__wrapped__
                profile = tecton_login(
                    "app.tecton.ai", manual=True, okta_session_token=None
                )
            print(who_am_i())
            return
        # Authenticate if not already authenticated
        profile = who_am_i()
        if not profile:
            tecton_login = tecton_cli.login.callback.__wrapped__
            profile = tecton_login(
                "app.tecton.ai", manual=True, okta_session_token=None
            )
        # Run all other commands via shell magic
        ipython = get_ipython()
        sh_magic = ipython.magics_manager.lsmagic()["cell"]["sh"]
        if isinstance(profile, ServiceAccountProfile):
            api_key = conf.get_or_none("TECTON_API_KEY")
            api_service = conf.get_or_none("API_SERVICE")
            ## TODO: Additional validation on the injected variables below should be done to mitigate the risk of a shell injection attack vector.  api_key, api_service and line should all have validation done.  line's validation might be as simple as checking for a semicolon, since that input comes from the user directly.  whereas API_KEY and API_SERVICE are from a config file that an attacker might modify.
            sh_magic(
                None,
                f"TECTON_API_KEY='{api_key}' API_SERVICE='{api_service}' tecton {line}; true",
            )
        else:
            sh_magic(None, f"tecton {line}; true")
        return

In [ ]:
setup_tecton_magic()

In [ ]:
%tecton whoami

In [ ]:
%tecton login

In [ ]:
%tecton login --when-needed

In [ ]:
dev_workspace = "ci-cd-example-dev"
test_workspace = "ci-cd-example-test"
prod_workspace = "ci-cd-example-prod"

workspaces = [dev_workspace, test_workspace, prod_workspace]

In [ ]:
%tecton workspace create $dev_workspace
%tecton init

In [ ]:
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregation
from datetime import datetime, timedelta

transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)
user = Entity(name="user", join_keys=["user_id"])


@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="spark_sql",
    aggregation_interval=timedelta(days=1),
    aggregations=[Aggregation(function="mean", column="amt", time_window=timedelta(days=1)),],
)
def user_transaction_metrics(transactions):
    return f"""
        SELECT user_id, timestamp, amt
        FROM {transactions}
        """

user_transaction_metrics.validate()

In [ ]:
start = datetime(2022, 1, 1)
end = datetime(2022, 2, 1)

df = user_transaction_metrics.get_historical_features(start_time=start, end_time=end).to_pandas()

display(df.head(5))

In [ ]:
%%writefile features.py
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregation
from datetime import datetime, timedelta

transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)
user = Entity(name="user", join_keys=["user_id"])


@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="spark_sql",
    aggregation_interval=timedelta(days=1),
    aggregations=[Aggregation(function="mean", column="amt", time_window=timedelta(days=1)),],
)
def user_transaction_metrics(transactions):
    return f"""
        SELECT user_id, timestamp, amt
        FROM {transactions}
        """

In [ ]:
%tecton apply -y

In [ ]:
%tecton workspace create $test_workspace --live
%tecton init

In [ ]:
%%writefile features.py
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregation
from datetime import datetime, timedelta

transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)
user = Entity(name="user", join_keys=["user_id"])


@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="spark_sql",
    aggregation_interval=timedelta(days=1),
    aggregations=[Aggregation(function="mean", column="amt", time_window=timedelta(days=1)),],
    offline=True,
    online=True,
    feature_start_time=datetime(2023, 1, 1)
)
def user_transaction_metrics(transactions):
    return f"""
        SELECT user_id, timestamp, amt
        FROM {transactions}
        """

In [ ]:
%tecton apply -y

In [ ]:
%tecton workspace create $prod_workspace --live
%tecton init

In [ ]:
%%writefile features.py
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregation
from datetime import datetime, timedelta

transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)
user = Entity(name="user", join_keys=["user_id"])


@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="spark_sql",
    aggregation_interval=timedelta(days=1),
    aggregations=[Aggregation(function="mean", column="amt", time_window=timedelta(days=1)),],
    offline=True,
    online=True,
    feature_start_time=datetime(2013, 1, 1)
)
def user_transaction_metrics(transactions):
    return f"""
        SELECT user_id, timestamp, amt
        FROM {transactions}
        """

In [ ]:
%tecton apply

In [ ]:
for workspace in workspaces:
  %tecton destroy --workspace $workspace -y